##### Copyright 2025 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
import pandas as pd
import json

# Dummy data for demonstration
data = {
    'product': ['A', 'B', 'A', 'C', 'B', 'A', 'C', 'A'],
    'sales': [100, 150, 120, 80, 200, 110, 90, 130],
    'region': ['East', 'West', 'East', 'North', 'South', 'West', 'North', 'East']
}
df = pd.DataFrame(data)

# Define a simple tool for the AI agent to interact with data
def query_data(query_string):
    """
    Executes a simple query on the DataFrame and returns the result.
    Example query_string: 'df[df["sales"] > 100].to_json()'
    """
    try:
        result = eval(query_string)
        # Convert numpy types to standard Python types for JSON serialization
        if isinstance(result, (pd.Series, pd.DataFrame)):
            return result.to_json()
        elif isinstance(result, list):
            return json.dumps(result)
        elif isinstance(result, (int, float, str, dict, bool)):
            return json.dumps(result)
        else:
            return json.dumps(str(result)) # Fallback for other types
    except Exception as e:
        return json.dumps({"error": str(e)})

# Simulate an LLM's ability to call tools
def simulate_llm_tool_call(user_query):
    if "total sales" in user_query.lower():
        return "query_data('df[\"sales\"].sum().item()')"
    elif "sales by product" in user_query.lower():
        return "query_data('df.groupby(\"product\")[\"sales\"].sum().to_json()')"
    elif "products in east region" in user_query.lower():
        return "query_data('df[df[\"region\"] == \"East\"][\"product\"].tolist()')"
    elif "average sales" in user_query.lower():
        return "query_data('df[\"sales\"].mean().item()')"
    elif "raw data" in user_query.lower():
        return "query_data('df.to_json(orient=\"records\")')"
    else:
        return None

def run_ai_agent(user_question):
    print(f"User: {user_question}")
    tool_call_string = simulate_llm_tool_call(user_question)

    if tool_call_string:
        print(f"Agent (simulated LLM): Decided to call tool with: {tool_call_string}")
        # Execute the tool call
        tool_name, args_str = tool_call_string.split('(', 1)
        args_str = args_str[:-1] # Remove trailing ')'

        # In a real scenario, you'd parse args_str more robustly
        # For this simple example, we know it's a single string argument
        arg = args_str.strip("'")

        if tool_name == "query_data":
            tool_result = query_data(arg)
            print(f"Tool (query_data) Result: {tool_result}")
            # Simulate LLM processing the tool result and generating a natural language response
            if "total sales" in user_question.lower():
                total_sales = json.loads(tool_result)
                print(f"Agent: The total sales are {total_sales}.")
            elif "sales by product" in user_question.lower():
                sales_by_product = json.loads(tool_result)
                print(f"Agent: Sales by product are: {sales_by_product}")
            elif "products in east region" in user_question.lower():
                products = json.loads(tool_result)
                print(f"Agent: Products in the East region are: {', '.join(products)}.")
            elif "average sales" in user_question.lower():
                avg_sales = json.loads(tool_result)
                print(f"Agent: The average sales are {avg_sales:.2f}.")
            elif "raw data" in user_question.lower():
                raw_data = json.loads(tool_result)
                print(f"Agent: Here is the raw data: {raw_data}")
            else:
                print(f"Agent: I processed the data. Here is the raw result: {tool_result}")
        else:
            print("Agent: Unknown tool called.")
    else:
        print("Agent: I cannot answer that question with my current tools.")

if __name__ == "__main__":
    print("\n--- Data Analyst AI Agent Simulation ---")
    run_ai_agent("What are the total sales?")
    print("\n")
    run_ai_agent("Show me sales by product.")
    print("\n")
    run_ai_agent("List products in the East region.")
    print("\n")
    run_ai_agent("What is the average sales?")
    print("\n")
    run_ai_agent("Show me the raw data.")
    print("\n")

# Gemini API: Prompting Quickstart

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Prompting.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

This notebook contains examples of how to write and run your first prompts with the Gemini API.

In [5]:
%pip install -U -q "google-genai>=1.4.0" # 1.4.0 is needed for chat history

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.3/257.3 kB 8.1 MB/s eta 0:00:00


### Set up your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see the [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) quickstart for an example.

In [6]:
from google.colab import userdata
from google import genai

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)

SecretNotFoundError: Secret GOOGLE_API_KEY does not exist.

## Select your model

Now select the model you want to use in this guide, either by selecting one in the list or writing it down. Keep in mind that some models, like the 2.5 ones are thinking models and thus take slightly more time to respond (cf. [thinking notebook](./Get_started_thinking.ipynb) for more details and in particular learn how to switch the thiking off).

In [ ]:
MODEL_ID = "gemini-2.5-flash" # @param ["gemini-2.5-flash-lite", "gemini-2.5-flash-lite-preview-09-2025", "gemini-2.5-flash", "gemini-2.5-flash-preview-09-2025", "gemini-2.5-pro"] {"allow-input":true, isTemplate: true}

## Run your first prompt

Use the `generate_content` method to generate responses to your prompts. You can pass text directly to generate_content, and use the `.text` property to get the text content of the response.

In [ ]:
from IPython.display import Markdown

response = client.models.generate_content(
    model=MODEL_ID,
    contents="Give me python code to sort a list"
)

display(Markdown(response.text))

## Use images in your prompt

Here you will download an image from a URL and pass that image in our prompt.

First, you download the image and load it with PIL:

In [ ]:
!curl -o image.jpg "https://storage.googleapis.com/generativeai-downloads/images/jetpack.jpg"

In [ ]:
import PIL.Image
img = PIL.Image.open('image.jpg')
img

In [ ]:
prompt = """
    This image contains a sketch of a potential product along with some notes.
    Given the product sketch, describe the product as thoroughly as possible based on what you
   see in the image, making sure to note all of the product features. Return output in json format:
   {description: description, features: [feature1, feature2, feature3, etc]}
"""

Then you can include the image in our prompt by just passing a list of items to `generate_content`.

In [ ]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents=[prompt, img],
)

print(response.text)

## Have a chat

The Gemini API enables you to have freeform conversations across multiple turns.

The [ChatSession](https://ai.google.dev/api/python/google/generativeai/ChatSession) class will store the conversation history for multi-turn interactions.

In [ ]:
chat = client.chats.create(model=MODEL_ID)

In [ ]:
response = chat.send_message(
    message="In one sentence, explain how a computer works to a young child."
)

print(response.text)

You can see the chat history:

In [ ]:
messages = chat.get_history()
for message in messages:
  print(f"{message.role}: {message.parts[0].text}")

You can keep sending messages to continue the conversation:

In [ ]:
response = chat.send_message("Okay, how about a more detailed explanation to a high schooler?")

print(response.text)

## Set the temperature

Every prompt you send to the model includes parameters that control how the model generates responses. Use a `types.GenerateContentConfig` to set these, or omit it to use the defaults.

Temperature controls the degree of randomness in token selection. Use higher values for more creative responses, and lower values for more deterministic responses.

Note: Although you can set the `candidate_count` in the generation_config, 2.0 and later models will only return a single candidate at the this time.

In [ ]:
from google.genai import types

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Give me a numbered list of cat facts.',
    config=types.GenerateContentConfig(
        max_output_tokens=2000,
        temperature=1.9,
        stop_sequences=['\n6'] # Limit to 5 facts.
    )
)

display(Markdown(response.text))

## Learn more

There's lots more to learn!

* For more fun prompts, check out [Market a Jetpack](https://github.com/google-gemini/cookbook/blob/main/examples/Market_a_Jet_Backpack.ipynb).
* Check out the [safety quickstart](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Safety.ipynb) next to learn about the Gemini API's configurable safety settings, and what to do if your prompt is blocked.
* For lots more details on using the Python SDK, check out the [get started notebook](./Get_started.ipynb) or the [documentation's quickstart](https://ai.google.dev/tutorials/python_quickstart).